- 해당 노트북 파일은 주피터 노트북을 활용해 작성되었습니다.

- 필요 패키지 불러오기

In [1]:
import pandas as pd

import requests
import json
from tqdm.notebook import tqdm

- 데이터 불러오기

In [2]:
school = pd.read_csv('./서울특별시 학교 기본정보.csv', encoding='cp949')
mart = pd.read_csv('./서울특별시 대규모점포 인허가 정보.csv', encoding='cp949')
subway = pd.read_csv('./서울교통공사_역주소 및 전화번호_20200715.csv', encoding='cp949')
commercial = pd.read_csv('./서울시 우리마을가게 상권분석서비스(상권영역).csv', encoding='cp949')

### 1. 지하철 좌표 추출

In [3]:
subway.head()

,연번,역번호,호선,역명,역전화번호,역주소
0,1,150,1,서울역,02-6110-1331,서울특별시 중구 세종대로 지하 2 (남대문로 5가)
1,2,151,1,시청,02-6110-1321,서울특별시 중구 세종대로 지하 101 (정동)
2,3,152,1,종각,02-6110-1311,서울특별시 종로구 종로 지하 55 (종로1가)
3,4,153,1,종로3가,02-6110-1301,서울특별시 종로구 종로 지하 129 (종로3가)
4,5,154,1,종로5가,02-6110-1291,서울특별시 종로구 종로 지하 216 (종로5가)


In [4]:
# 도로명 주소로 좌표를 반환할 수 있는 URL
KAKAO_URL = "https://dapi.kakao.com/v2/local/search/address.json"

# Kakao 개인 REST APP KEY
with open("./KakaoApiKey.txt", "r") as api_key:
    REST_APP_KEY = api_key.read()

In [5]:
def kakao_geocode(address=None):
    
    url = "{}?query={}".format(KAKAO_URL, address)
    
    def json_request(url = url):
        
        # Headers 지정
        headers = {"Authorization": "KakaoAK {}".format(REST_APP_KEY)}
        
        # URL 정보 불러오기
        reponse = requests.get(url = url, headers = headers)
        
        return reponse.text
    
    try:
        json_response = json_request()
        json_data = json.loads(json_response)
        
        # 도로명 주소으로부터 좌표를 얻어오는 과정에서 'x', 'y' 결과만 가져오도록 한다.
        # 도로명 주소이므로 road_address에서 x, y를 추출해온다.
        
        json_doc = json_data.get("documents")[0]["road_address"]
        
        # '경도(lon)' 추출
        lon = json_doc.get("x")
        
        # '위도(lat)' 추출
        lat = json_doc.get("y")
        
    except:
        lon = "획득불가"
        lat = "획득불가"
    
    return lon, lat

- 도로명 주소별 좌표 부여

In [6]:
# 'kakao_geocode' 함수를 사용해 '경도' 값을 추출해주는 함수 생성
def get_lon(address=None):
    
    # 'kakao_geocode' 함수를 사용하여 반환값 저장
    lon, lat = kakao_geocode(address)
    
    return lon

# 'kakao_geocode' 함수를 사용해 '위도' 값을 추출해주는 함수 생성
def get_lat(address=None):
    
    # 'kakao_geocode' 함수를 사용하여 반환값 저장
    lon, lat = kakao_geocode(address)
    
    return lat

In [7]:
print("Shape of data *Before Create* :", subway.shape, "\n")

# 반복문을 통해 데이터 추출
for index in tqdm(subway.index, desc = "카카오 API 도로명 주소로 좌표 추출"):
    
    address = str(subway.loc[index, "역주소"])
        
    lon = get_lon(address)
    lat = get_lat(address)
    
    subway.loc[index, "lon"] = lon
    subway.loc[index, "lat"] = lat

print()
print("Shape of data *After Create* :", subway.shape, "\n")
display(subway.head())

Shape of data *Before Create* : (292, 6) 



카카오 API 도로명 주소로 좌표 추출:   0%|          | 0/292 [00:00<?, ?it/s]


Shape of data *After Create* : (292, 8) 



,연번,역번호,호선,역명,역전화번호,역주소,lon,lat
0,1,150,1,서울역,02-6110-1331,서울특별시 중구 세종대로 지하 2 (남대문로 5가),126.972554606456,37.5571594835988
1,2,151,1,시청,02-6110-1321,서울특별시 중구 세종대로 지하 101 (정동),126.976983237511,37.5654385176408
2,3,152,1,종각,02-6110-1311,서울특별시 종로구 종로 지하 55 (종로1가),126.983239678933,37.5702140381269
3,4,153,1,종로3가,02-6110-1301,서울특별시 종로구 종로 지하 129 (종로3가),126.99202930223,37.5704266025836
4,5,154,1,종로5가,02-6110-1291,서울특별시 종로구 종로 지하 216 (종로5가),127.001916647342,37.5709075371016


- 획득하지 못한 좌표값들 처리

In [138]:
subway[subway['lon']=='획득불가']

,연번,역번호,호선,역명,역전화번호,역주소,lon,lat
73,74,321,3,충무로,02-6110-4231,서울특별시 중구 퇴계로 지하 199 (필동2가),획득불가,획득불가
109,110,423,4,충무로,02-6110-4231,서울특별시 중구 퇴계로 지하 199 (필동2가),획득불가,획득불가
116,117,430,4,이촌,02-6110-4301,서울특별시 용산구 용산동5가 17,획득불가,획득불가
195,196,2634,6,약수,02-6311-6331,서울특별시 중구 다산로 지하 155 (신당동),획득불가,획득불가
222,223,2722,7,상봉,02-6311-7201,서울특별시 중랑구 망우로 지하 297,획득불가,획득불가
290,291,4137,9,둔촌오륜,02-2656-0937,서울특별시 송파구 강동대로 327,획득불가,획득불가


- 충무로역

In [139]:
subway.loc[[73, 109], '역주소'] = '서울 중구 퇴계로 지하 214'

In [140]:
subway.loc[[73, 109], '역주소']

73     서울 중구 퇴계로 지하 214
109    서울 중구 퇴계로 지하 214
Name: 역주소, dtype: object

In [141]:
lon = get_lon(subway.loc[73, '역주소'])
lat = get_lat(subway.loc[73, '역주소'])

subway.loc[[73, 109], 'lon'] = lon
subway.loc[[73, 109], 'lat'] = lat


In [142]:
subway.loc[[73, 109], :]

,연번,역번호,호선,역명,역전화번호,역주소,lon,lat
73,74,321,3,충무로,02-6110-4231,서울 중구 퇴계로 지하 214,126.994207448462,37.5612890978954
109,110,423,4,충무로,02-6110-4231,서울 중구 퇴계로 지하 214,126.994207448462,37.5612890978954


- 이촌역

In [143]:
subway.loc[116, '역주소'] = '서울 용산구 서빙고로 지하 83'

In [144]:
lon = get_lon(subway.loc[116, '역주소'])
lat = get_lat(subway.loc[116, '역주소'])

subway.loc[116, 'lon'] = lon
subway.loc[116, 'lat'] = lat

In [145]:
subway.loc[116, :]

연번                     117
역번호                    430
호선                       4
역명                      이촌
역전화번호         02-6110-4301
역주소      서울 용산구 서빙고로 지하 83
lon       126.973683151228
lat        37.522520254623
Name: 116, dtype: object

- 약수역

In [146]:
subway.loc[195, '역주소'] = '서울 중구 다산로 지하 122'

In [147]:
lon = get_lon(subway.loc[195, '역주소'])
lat = get_lat(subway.loc[195, '역주소'])

subway.loc[195, 'lon'] = lon
subway.loc[195, 'lat'] = lat

In [148]:
subway.loc[195, :]

연번                    196
역번호                  2634
호선                      6
역명                     약수
역전화번호        02-6311-6331
역주소      서울 중구 다산로 지하 122
lon      127.010822526737
lat      37.5545595558163
Name: 195, dtype: object

- 상봉역

In [149]:
subway.loc[222, '역주소'] = '서울 중랑구 망우로 297'

In [150]:
lon = get_lon(subway.loc[222, '역주소'])
lat = get_lat(subway.loc[222, '역주소'])

subway.loc[222, 'lon'] = lon
subway.loc[222, 'lat'] = lat

In [151]:
subway.loc[222, :]

연번                    223
역번호                  2722
호선                      7
역명                     상봉
역전화번호        02-6311-7201
역주소        서울 중랑구 망우로 297
lon      127.085436765164
lat      37.5967137844087
Name: 222, dtype: object

- 둔촌오륜역

In [152]:
subway.loc[290, '역주소'] = '서울 강동구 강동대로 지하 303'

In [153]:
lon = get_lon(subway.loc[290, '역주소'])
lat = get_lat(subway.loc[290, '역주소'])

subway.loc[290, 'lon'] = lon
subway.loc[290, 'lat'] = lat

In [154]:
subway.loc[290, :]

연번                      291
역번호                    4137
호선                        9
역명                     둔촌오륜
역전화번호          02-2656-0937
역주소      서울 강동구 강동대로 지하 303
lon        127.138427332482
lat        37.5194242486054
Name: 290, dtype: object

In [155]:
subway[subway['lon']=='획득불가']

,연번,역번호,호선,역명,역전화번호,역주소,lon,lat


- 저장

In [157]:
subway.to_csv('./subway_xy.csv', index=False, encoding='euc-kr')

### 2. 학교 좌표 추출

In [5]:
school.head()

,학교종류명,설립구분,표준학교코드,학교명,영문학교명,관할조직명,도로명우편번호,도로명주소,도로명상세주소,전화번호,...,주야구분명,설립일자,개교기념일,시도교육청코드,시도교육청명,소재지명,주야과정,계열명,학과명,적재일시
0,각종학교(중),사립,7134155,선화예술중학교,Sunhwa Arts Middle School,서울특별시성동광진교육지원청,4991.0,서울특별시 광진구 천호대로 664,/ 선화예술중고등학교 (능동),02-2204-1100,...,주간,19731201,19730705,B10,서울특별시교육청,서울특별시,NaN,NaN,NaN,20211108
1,초등학교,공립,7134150,서울숭신초등학교,Seoul Soongshin Elementary School,서울특별시성동광진교육지원청,4702.0,서울특별시 성동구 마장로 161,(하왕십리동),02-2252-5950,...,주간,19590403,19590403,B10,서울특별시교육청,서울특별시,NaN,NaN,NaN,20211108
2,중학교,공립,7134142,행당중학교,Haengdang Middle School,서울특별시성동광진교육지원청,4764.0,서울특별시 성동구 왕십리로 189,(행당동/행당중학교),02-2292-2721,...,주간,19680806,19681002,B10,서울특별시교육청,서울특별시,NaN,NaN,NaN,20211108
3,중학교,사립,7134141,한양대학교사범대학부속중학교,Hanyang University Middle School,서울특별시성동광진교육지원청,4761.0,서울특별시 성동구 마조로 42,(사근동/한양사대부속중?고등학교),02-2200-3700,...,주간,19600118,19600118,B10,서울특별시교육청,서울특별시,NaN,NaN,NaN,20211108
4,중학교,공립,7134140,자양중학교,Jayang Middle School,서울특별시성동광진교육지원청,5069.0,서울특별시 광진구 뚝섬로41길 33,(자양동/서울자양중학교),02-446-0365,...,주간,19840301,19840301,B10,서울특별시교육청,서울특별시,NaN,NaN,NaN,20211108


In [6]:
school[school['도로명주소'].isnull()]

,학교종류명,설립구분,표준학교코드,학교명,영문학교명,관할조직명,도로명우편번호,도로명주소,도로명상세주소,전화번호,...,주야구분명,설립일자,개교기념일,시도교육청코드,시도교육청명,소재지명,주야과정,계열명,학과명,적재일시
3113,고등학교,사립,7010184,서울국악예술고등학교,Seoul Traditional Arts High School,서울특별시교육청,NaN,NaN,NaN,02-896-1095,...,주간,19600513,19600513,B10,서울특별시교육청,서울특별시,주간,일반계,타악과,20211108
3114,고등학교,사립,7010184,서울국악예술고등학교,Seoul Traditional Arts High School,서울특별시교육청,NaN,NaN,NaN,02-896-1095,...,주간,19600513,19600513,B10,서울특별시교육청,서울특별시,주간,일반계,작곡이론과,20211108
3115,고등학교,사립,7010184,서울국악예술고등학교,Seoul Traditional Arts High School,서울특별시교육청,NaN,NaN,NaN,02-896-1095,...,주간,19600513,19600513,B10,서울특별시교육청,서울특별시,주간,예술계,기악과,20211108
3116,고등학교,사립,7010184,서울국악예술고등학교,Seoul Traditional Arts High School,서울특별시교육청,NaN,NaN,NaN,02-896-1095,...,주간,19600513,19600513,B10,서울특별시교육청,서울특별시,주간,예술계,작곡이론과,20211108
3117,고등학교,사립,7010184,서울국악예술고등학교,Seoul Traditional Arts High School,서울특별시교육청,NaN,NaN,NaN,02-896-1095,...,주간,19600513,19600513,B10,서울특별시교육청,서울특별시,주간,예술계,타악과,20211108
3118,고등학교,사립,7010184,서울국악예술고등학교,Seoul Traditional Arts High School,서울특별시교육청,NaN,NaN,NaN,02-896-1095,...,주간,19600513,19600513,B10,서울특별시교육청,서울특별시,주간,일반계,무용과,20211108
3119,고등학교,사립,7010184,서울국악예술고등학교,Seoul Traditional Arts High School,서울특별시교육청,NaN,NaN,NaN,02-896-1095,...,주간,19600513,19600513,B10,서울특별시교육청,서울특별시,주간,일반계,음악연극과,20211108
3120,고등학교,사립,7010184,서울국악예술고등학교,Seoul Traditional Arts High School,서울특별시교육청,NaN,NaN,NaN,02-896-1095,...,주간,19600513,19600513,B10,서울특별시교육청,서울특별시,주간,일반계,성악과,20211108
3121,고등학교,사립,7010184,서울국악예술고등학교,Seoul Traditional Arts High School,서울특별시교육청,NaN,NaN,NaN,02-896-1095,...,주간,19600513,19600513,B10,서울특별시교육청,서울특별시,주간,예술계,무용과,20211108
3122,고등학교,사립,7010184,서울국악예술고등학교,Seoul Traditional Arts High School,서울특별시교육청,NaN,NaN,NaN,02-896-1095,...,주간,19600513,19600513,B10,서울특별시교육청,서울특별시,주간,예술계,성악과,20211108


- 도로명주소가 없는 학교는 서울국악예술고등학교
- 직접 주소 입력

In [7]:
no_index = school[school['도로명주소'].isnull()].index
school.loc[no_index, '도로명주소'] = '서울 강남구 개포로22길 65'

In [8]:
school['도로명주소'].isnull().sum()

0

In [9]:
school[school['학교명'] == '서울국악예술고등학교']

,학교종류명,설립구분,표준학교코드,학교명,영문학교명,관할조직명,도로명우편번호,도로명주소,도로명상세주소,전화번호,...,주야구분명,설립일자,개교기념일,시도교육청코드,시도교육청명,소재지명,주야과정,계열명,학과명,적재일시
3113,고등학교,사립,7010184,서울국악예술고등학교,Seoul Traditional Arts High School,서울특별시교육청,NaN,서울 강남구 개포로22길 65,NaN,02-896-1095,...,주간,19600513,19600513,B10,서울특별시교육청,서울특별시,주간,일반계,타악과,20211108
3114,고등학교,사립,7010184,서울국악예술고등학교,Seoul Traditional Arts High School,서울특별시교육청,NaN,서울 강남구 개포로22길 65,NaN,02-896-1095,...,주간,19600513,19600513,B10,서울특별시교육청,서울특별시,주간,일반계,작곡이론과,20211108
3115,고등학교,사립,7010184,서울국악예술고등학교,Seoul Traditional Arts High School,서울특별시교육청,NaN,서울 강남구 개포로22길 65,NaN,02-896-1095,...,주간,19600513,19600513,B10,서울특별시교육청,서울특별시,주간,예술계,기악과,20211108
3116,고등학교,사립,7010184,서울국악예술고등학교,Seoul Traditional Arts High School,서울특별시교육청,NaN,서울 강남구 개포로22길 65,NaN,02-896-1095,...,주간,19600513,19600513,B10,서울특별시교육청,서울특별시,주간,예술계,작곡이론과,20211108
3117,고등학교,사립,7010184,서울국악예술고등학교,Seoul Traditional Arts High School,서울특별시교육청,NaN,서울 강남구 개포로22길 65,NaN,02-896-1095,...,주간,19600513,19600513,B10,서울특별시교육청,서울특별시,주간,예술계,타악과,20211108
3118,고등학교,사립,7010184,서울국악예술고등학교,Seoul Traditional Arts High School,서울특별시교육청,NaN,서울 강남구 개포로22길 65,NaN,02-896-1095,...,주간,19600513,19600513,B10,서울특별시교육청,서울특별시,주간,일반계,무용과,20211108
3119,고등학교,사립,7010184,서울국악예술고등학교,Seoul Traditional Arts High School,서울특별시교육청,NaN,서울 강남구 개포로22길 65,NaN,02-896-1095,...,주간,19600513,19600513,B10,서울특별시교육청,서울특별시,주간,일반계,음악연극과,20211108
3120,고등학교,사립,7010184,서울국악예술고등학교,Seoul Traditional Arts High School,서울특별시교육청,NaN,서울 강남구 개포로22길 65,NaN,02-896-1095,...,주간,19600513,19600513,B10,서울특별시교육청,서울특별시,주간,일반계,성악과,20211108
3121,고등학교,사립,7010184,서울국악예술고등학교,Seoul Traditional Arts High School,서울특별시교육청,NaN,서울 강남구 개포로22길 65,NaN,02-896-1095,...,주간,19600513,19600513,B10,서울특별시교육청,서울특별시,주간,예술계,무용과,20211108
3122,고등학교,사립,7010184,서울국악예술고등학교,Seoul Traditional Arts High School,서울특별시교육청,NaN,서울 강남구 개포로22길 65,NaN,02-896-1095,...,주간,19600513,19600513,B10,서울특별시교육청,서울특별시,주간,예술계,성악과,20211108


In [10]:
print("Shape of data *Before Create* :", school.shape, "\n")

# 반복문을 통해 데이터 추출
for index in tqdm(school.index, desc = "카카오 API 도로명 주소로 좌표 추출"):
    
    address = str(school.loc[index, "도로명주소"])
        
    lon = get_lon(address)
    lat = get_lat(address)
    
    school.loc[index, "lon"] = lon
    school.loc[index, "lat"] = lat

print()
print("Shape of data *After Create* :", school.shape, "\n")
display(school.head())

Shape of data *Before Create* : (3763, 28) 



카카오 API 도로명 주소로 좌표 추출:   0%|          | 0/3763 [00:00<?, ?it/s]

NameError: name 'get_lon' is not defined

- 획득하지 못한 좌표값들 처리

In [199]:
school[school['lon']=='획득불가']

,학교종류명,설립구분,표준학교코드,학교명,영문학교명,관할조직명,도로명우편번호,도로명주소,도로명상세주소,전화번호,...,개교기념일,시도교육청코드,시도교육청명,소재지명,주야과정,계열명,학과명,적재일시,lon,lat
311,초등학교,공립,7121320,서울안암초등학교,Seoul Anam Elementary School,서울특별시성북강북교육지원청,2850.0,서울특별시 성북구 인촌로7가길 39,(안암동2가/서울안암초등학교),02-926-4426,...,19600607,B10,서울특별시교육청,서울특별시,NaN,NaN,NaN,20211108,획득불가,획득불가
426,초등학교,공립,7091371,서울개포초등학교,Seoul Kaepo Elementary School,서울특별시강남서초교육지원청,6331.0,서울특별시 강남구 영동대로3길 45,(개포동/개포초등학교),02-445-9321~3,...,19830510,B10,서울특별시교육청,서울특별시,NaN,NaN,NaN,20211108,획득불가,획득불가


- 서울안암초등학교

In [200]:
school.loc[311, '도로명주소'] = '서울 성북구 고려대로7가길 39'

In [202]:
lon = get_lon(school.loc[311, '도로명주소'])
lat = get_lat(school.loc[311, '도로명주소'])

school.loc[311, 'lon'] = lon
school.loc[311, 'lat'] = lat

In [203]:
school.loc[311, :]

학교종류명                                  초등학교
설립구분                                     공립
표준학교코드                              7121320
학교명                                서울안암초등학교
영문학교명          Seoul Anam Elementary School
관할조직명                        서울특별시성북강북교육지원청
도로명우편번호                              2850.0
도로명주소                     서울 성북구 고려대로7가길 39
도로명상세주소                    (안암동2가/서울안암초등학교)
전화번호                            02-926-4426
홈페이지주소                http://www.anam.es.kr
팩스번호                            02-926-1802
남녀공학구분명                                남여공학
고등학교구분명                                 NaN
산업체특별학급존재여부                               N
고등학교일반실업구분명                             일반계
특수목적고등학교계열명                             NaN
입시전후기구분명                                 전기
주야구분명                                    주간
설립일자                               19560324
개교기념일                              19600607
시도교육청코드                                 B10
시도교육청명                          

- 서울개포초등학교 --> 폐교

In [204]:
school.loc[426, :]

학교종류명                                   초등학교
설립구분                                      공립
표준학교코드                               7091371
학교명                                 서울개포초등학교
영문학교명          Seoul Kaepo Elementary School
관할조직명                         서울특별시강남서초교육지원청
도로명우편번호                               6331.0
도로명주소                    서울특별시 강남구 영동대로3길 45
도로명상세주소                         (개포동/개포초등학교)
전화번호                           02-445-9321~3
홈페이지주소                http://www.kaepo.es.kr
팩스번호                             02-445-9328
남녀공학구분명                                 남여공학
고등학교구분명                                  NaN
산업체특별학급존재여부                                N
고등학교일반실업구분명                              일반계
특수목적고등학교계열명                              NaN
입시전후기구분명                                  전기
주야구분명                                     주간
설립일자                                19830510
개교기념일                               19830510
시도교육청코드                                  B10
시도교육청명    

In [205]:
school

(3763, 30)

In [217]:
school_xy = school.drop(index = 426, axis = 0).reset_index(drop = True)

In [218]:
school_xy[school_xy['lon']=='획득불가']

,학교종류명,설립구분,표준학교코드,학교명,영문학교명,관할조직명,도로명우편번호,도로명주소,도로명상세주소,전화번호,...,개교기념일,시도교육청코드,시도교육청명,소재지명,주야과정,계열명,학과명,적재일시,lon,lat


In [220]:
school_xy.to_csv('./school_xy.csv', encoding='euc-kr', index=False)

### 3. 대규모점포

- 좌표값 존재 but 변환 필요

In [221]:
mart.head()

,개방자치단체코드,관리번호,인허가일자,인허가취소일자,영업상태코드,영업상태명,상세영업상태코드,상세영업상태명,폐업일자,휴업시작일자,...,도로명주소,도로명우편번호,사업장명,최종수정일자,데이터갱신구분,데이터갱신일자,업태구분명,좌표정보(X),좌표정보(Y),점포구분명
0,3000000,2019300020207500001,20190917,NaN,1,영업/정상,1,정상영업,NaN,NaN,...,서울특별시 종로구 삼일대로 428 낙원상가 506호 (낙원동),3140.0,낙원상가,20190925150602,U,2019-09-27 02:40:00.0,그 밖의 대규모점포,198838.912966,452214.210962,대규모점포
1,3000000,2019300020207500002,20190917,NaN,1,영업/정상,1,정상영업,NaN,NaN,...,서울특별시 종로구 삼일대로 428 낙원상가 (낙원동),3140.0,낙원상가,20190925150506,I,2019-09-27 02:22:37.0,그 밖의 대규모점포,198838.912966,452214.210962,대규모점포
2,3000000,2003300012907500001,20030304,NaN,1,영업/정상,1,정상영업,NaN,NaN,...,NaN,NaN,광장주식회사,20070801173946,I,2018-08-31 23:59:59.0,시장,199837.414267,451974.128468,NaN
3,3030000,2021303010307500001,20130219,NaN,1,영업/정상,1,정상영업,NaN,NaN,...,서울특별시 성동구 왕십리로 109 성수만세주유소 (성수동1가),4768.0,홈플러스익스프레스 뚝섬2점,20210614085250,U,2020-12-05 23:06:00.0,구분없음,203841.225548,449475.540141,NaN
4,3000000,2004300007607500003,20040430,NaN,1,영업/정상,1,정상영업,NaN,NaN,...,NaN,NaN,영풍문고,20070721102756,I,2018-08-31 23:59:59.0,대형마트,198351.818737,451921.603026,NaN


- 정상적으로 영업을 하고 있는 점포만 다룰 것

In [236]:
mart_active = mart[mart['영업상태명'] == '영업/정상'].reset_index(drop=True)

In [237]:
mart_active

,개방자치단체코드,관리번호,인허가일자,인허가취소일자,영업상태코드,영업상태명,상세영업상태코드,상세영업상태명,폐업일자,휴업시작일자,...,도로명주소,도로명우편번호,사업장명,최종수정일자,데이터갱신구분,데이터갱신일자,업태구분명,좌표정보(X),좌표정보(Y),점포구분명
0,3000000,2019300020207500001,20190917,NaN,1,영업/정상,1,정상영업,NaN,NaN,...,서울특별시 종로구 삼일대로 428 낙원상가 506호 (낙원동),3140.0,낙원상가,20190925150602,U,2019-09-27 02:40:00.0,그 밖의 대규모점포,198838.912966,452214.210962,대규모점포
1,3000000,2019300020207500002,20190917,NaN,1,영업/정상,1,정상영업,NaN,NaN,...,서울특별시 종로구 삼일대로 428 낙원상가 (낙원동),3140.0,낙원상가,20190925150506,I,2019-09-27 02:22:37.0,그 밖의 대규모점포,198838.912966,452214.210962,대규모점포
2,3000000,2003300012907500001,20030304,NaN,1,영업/정상,1,정상영업,NaN,NaN,...,NaN,NaN,광장주식회사,20070801173946,I,2018-08-31 23:59:59.0,시장,199837.414267,451974.128468,NaN
3,3030000,2021303010307500001,20130219,NaN,1,영업/정상,1,정상영업,NaN,NaN,...,서울특별시 성동구 왕십리로 109 성수만세주유소 (성수동1가),4768.0,홈플러스익스프레스 뚝섬2점,20210614085250,U,2020-12-05 23:06:00.0,구분없음,203841.225548,449475.540141,NaN
4,3000000,2004300007607500003,20040430,NaN,1,영업/정상,1,정상영업,NaN,NaN,...,NaN,NaN,영풍문고,20070721102756,I,2018-08-31 23:59:59.0,대형마트,198351.818737,451921.603026,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
702,3140000,2011314011407500002,20110630,NaN,1,영업/정상,1,정상영업,NaN,NaN,...,서울특별시 양천구 중앙로 255-1 (신정동),NaN,홈플러스(주)익스프레스 신정점,20210610181338,U,2020-12-05 23:02:00.0,구분없음,186934.147693,446336.680482,NaN
703,3060000,2011306009507500002,20110809,NaN,1,영업/정상,1,정상영업,NaN,NaN,...,서울특별시 중랑구 면목로 357 (면목동),131831.0,홈플러스(주)익스프레스 면목점,20210611132236,U,2020-12-05 23:03:00.0,그 밖의 대규모점포,207708.364766,453525.741162,NaN
704,3060000,2006306009507500002,20061026,NaN,1,영업/정상,1,정상영업,NaN,NaN,...,서울특별시 중랑구 사가정로 332 (면목동),131822.0,홈플러스(주)면목점,20210611132706,U,2020-12-05 23:03:00.0,복합쇼핑몰,207089.216591,453074.476764,NaN
705,3060000,2006306009507500001,20060828,NaN,1,영업/정상,1,정상영업,NaN,NaN,...,서울특별시 중랑구 신내로 201 (신내동),131872.0,홈플러스 신내점,20210611132320,U,2020-12-05 23:03:00.0,대형마트,208208.786685,457054.793656,NaN


#### (1) 좌표정보 없음

In [303]:
mart_active['좌표정보(X)'].isnull().sum()

50

In [304]:
mart_active[mart_active['좌표정보(X)'].isnull()]['지번주소'].isnull().sum()

3

- 좌표정보는 없지만 대부분 지번주소는 보유

In [305]:
no_mart = mart_active[mart_active['좌표정보(X)'].isnull()].reset_index(drop=True)

In [306]:
no_mart[['지번주소', '사업장명']]

,지번주소,사업장명
0,서울특별시 종로구 창신동 436번지 79 호,동대문신발도매상가
1,호,종로세운상가
2,NaN,현대시티타워
3,NaN,지에스수퍼마켓 중구만리점
4,서울특별시 성북구 돈암동 609번지 1 호,동소문스카이프라자
5,서울특별시 성북구 길음동 540번지 2 호,길음시장
6,서울특별시 성북구 길음동 535-8535-26540-2540-7,길음시장
7,서울특별시 성북구 길음동 535번지8호 535번지26호 540번지2호,길음시장
8,서울특별시 성북구 보문동4가 1번지 1 호,보문시장
9,서울특별시 성북구 하월곡동 88번지 345호,미아시장


- 결측값: 3개, 이상값: 1개

In [307]:
no_mart.loc[1, '지번주소'] = '서울 종로구 장사동 116-4'
no_mart.loc[2, '지번주소'] = '서울 중구 을지로6가 17-2'
no_mart.loc[3, '지번주소'] = '서울 중구 만리동2가 111-4'
no_mart.loc[48, '지번주소'] = '서울 서초구 잠원동 57-20'

In [308]:
# 도로명 주소로 좌표를 반환할 수 있는 URL
KAKAO_URL = "https://dapi.kakao.com/v2/local/search/address.json"

# Kakao 개인 REST APP KEY
with open("./KakaoApiKey.txt", "r") as api_key:
    REST_APP_KEY = api_key.read()

In [309]:
def kakao_geocode(address=None):
    
    url = "{}?query={}".format(KAKAO_URL, address)
    
    def json_request(url = url):
        
        # Headers 지정
        headers = {"Authorization": "KakaoAK {}".format(REST_APP_KEY)}
        
        # URL 정보 불러오기
        reponse = requests.get(url = url, headers = headers)
        
        return reponse.text
    
    try:
        json_response = json_request()
        json_data = json.loads(json_response)
        
        # 지번 주소으로부터 좌표를 얻어오는 과정에서 'x', 'y' 결과만 가져오도록 한다.
        # 지번 주소이므로 road_address에서 x, y를 추출해온다.
        
        json_doc = json_data.get("documents")[0]["address"]
        
        # '경도(lon)' 추출
        lon = json_doc.get("x")
        
        # '위도(lat)' 추출
        lat = json_doc.get("y")
        
    except:
        lon = "획득불가"
        lat = "획득불가"
    
    return lon, lat

In [310]:
# 'kakao_geocode' 함수를 사용해 '경도' 값을 추출해주는 함수 생성
def get_lon(address=None):
    
    # 'kakao_geocode' 함수를 사용하여 반환값 저장
    lon, lat = kakao_geocode(address)
    
    return lon

# 'kakao_geocode' 함수를 사용해 '위도' 값을 추출해주는 함수 생성
def get_lat(address=None):
    
    # 'kakao_geocode' 함수를 사용하여 반환값 저장
    lon, lat = kakao_geocode(address)
    
    return lat

In [312]:
print("Shape of data *Before Create* :", no_mart.shape, "\n")

# 반복문을 통해 데이터 추출
for index in tqdm(no_mart.index, desc = "카카오 API 도로명 주소로 좌표 추출"):
    
    address = str(no_mart.loc[index, "지번주소"])
        
    lon = get_lon(address)
    lat = get_lat(address)
    
    no_mart.loc[index, "lon"] = lon
    no_mart.loc[index, "lat"] = lat

print()
print("Shape of data *After Create* :", no_mart.shape, "\n")
display(no_mart.head())

Shape of data *Before Create* : (50, 26) 



카카오 API 도로명 주소로 좌표 추출:   0%|          | 0/50 [00:00<?, ?it/s]


Shape of data *After Create* : (50, 28) 



,개방자치단체코드,관리번호,인허가일자,인허가취소일자,영업상태코드,영업상태명,상세영업상태코드,상세영업상태명,폐업일자,휴업시작일자,...,사업장명,최종수정일자,데이터갱신구분,데이터갱신일자,업태구분명,좌표정보(X),좌표정보(Y),점포구분명,lon,lat
0,3000000,2004300007607500017,20040630,NaN,1,영업/정상,1,정상영업,NaN,NaN,...,동대문신발도매상가,20070721102756,I,2018-08-31 23:59:59.0,그 밖의 대규모점포,NaN,NaN,NaN,127.010641054211,37.5700690573243
1,3000000,2005300007607500001,20050127,NaN,1,영업/정상,1,정상영업,NaN,NaN,...,종로세운상가,20070721102756,I,2018-08-31 23:59:59.0,대형마트,NaN,NaN,NaN,126.995238594891,37.5692909807898
2,3010000,2014301013007500017,19960831,NaN,1,영업/정상,1,정상영업,NaN,NaN,...,현대시티타워,20170508135141,I,2018-08-31 23:59:59.0,쇼핑센터,NaN,NaN,대규모점포,127.007732349005,37.5686434582653
3,3010000,2017301013007500002,20171031,NaN,1,영업/정상,1,정상영업,NaN,NaN,...,지에스수퍼마켓 중구만리점,20180123132504,I,2018-08-31 23:59:59.0,구분없음,NaN,NaN,준대규모점포,126.965270178104,37.5541725606871
4,3070000,2000307009907500015,19951116,NaN,1,영업/정상,1,정상영업,NaN,NaN,...,동소문스카이프라자,20070707104737,I,2018-08-31 23:59:59.0,그 밖의 대규모점포,NaN,NaN,NaN,127.010321322145,37.5941783218836


- 지번주소로도 좌표값을 얻을 수 없는 점포는 24개 --> 수작업으로 지번값 넣어서 좌표 추출

In [313]:
no_mart[no_mart['lon']=='획득불가'].loc[:,['지번주소', '사업장명']]

,지번주소,사업장명
6,서울특별시 성북구 길음동 535-8535-26540-2540-7,길음시장
7,서울특별시 성북구 길음동 535번지8호 535번지26호 540번지2호,길음시장
8,서울특별시 성북구 보문동4가 1번지 1 호,보문시장
9,서울특별시 성북구 하월곡동 88번지 345호,미아시장
10,서울특별시 성북구 석관동 270호,석관시장
13,서울특별시 은평구 진관내동 500번지 2 호,(주)미소종합유통
14,서울특별시 은평구 수색동 205번지 21 호,수일시장
15,서울특별시 은평구 응암동 300번지 11 호,양지시장
16,서울특별시 은평구 대조동 14번지 24호,불광시장(주)
17,서울특별시 은평구 대조동 14번지 37 호,대조시장


In [314]:
no_mart.loc[[6, 7], '지번주소'] = '서울 성북구 길음동 540-2'
no_mart.loc[8, '지번주소'] = '서울 성북구 보문동1가 206'
no_mart.loc[9, '지번주소'] = '서울 성북구 하월곡동 88-344'
no_mart.loc[10, '지번주소'] = '서울 성북구 석관동 270-1'
no_mart.loc[14, '지번주소'] = '서울 은평구 수색동 368-4'
no_mart.loc[15, '지번주소'] = '서울 은평구 응암동 754-1'
no_mart.loc[16, '지번주소'] = '서울 은평구 대조동 14-62'
no_mart.loc[17, '지번주소'] = '서울 은평구 대조동 12-3'
no_mart.loc[18, '지번주소'] = '서울 양천구 목동 727-1'
no_mart.loc[19, '지번주소'] = '서울 마포구 망원동 480-3'
no_mart.loc[20, '지번주소'] = '서울 양천구 신월동 446-1'
no_mart.loc[21, '지번주소'] = '서울 양천구 목동 602-1'
no_mart.loc[22, '지번주소'] = '서울 양천구 신월동 105-1'
no_mart.loc[24, '지번주소'] = '서울 강서구 방화동 619-36'
no_mart.loc[40, '지번주소'] = '서울 영등포구 도림동 176-9'
no_mart.loc[41, '지번주소'] = '서울 영등포구 신길동 259-1'
no_mart.loc[42, '지번주소'] = '서울 영등포구 대림동 1072-30'
no_mart.loc[[37, 44, 45], '지번주소'] = '서울 영등포구 영등포동3가 32'
no_mart.loc[46, '지번주소'] = '서울 영등포구 영등포동5가 19-1'
no_mart.loc[47, '지번주소'] = '서울 영등포구 문래동3가 10'

- (주)미소종합유통은 주소 정보가 불확실하여 제거 --> 95년 개업, 정보X

In [315]:
no_mart = no_mart.drop(index=13, axis=0).reset_index(drop=True)

In [316]:
print("Shape of data *Before Create* :", no_mart.shape, "\n")

# 반복문을 통해 데이터 추출
for index in tqdm(no_mart.index, desc = "카카오 API 도로명 주소로 좌표 추출"):
    
    address = str(no_mart.loc[index, "지번주소"])
        
    lon = get_lon(address)
    lat = get_lat(address)
    
    no_mart.loc[index, "lon"] = lon
    no_mart.loc[index, "lat"] = lat

print()
print("Shape of data *After Create* :", no_mart.shape, "\n")
display(no_mart.head())

Shape of data *Before Create* : (49, 28) 



카카오 API 도로명 주소로 좌표 추출:   0%|          | 0/49 [00:00<?, ?it/s]


Shape of data *After Create* : (49, 28) 



,개방자치단체코드,관리번호,인허가일자,인허가취소일자,영업상태코드,영업상태명,상세영업상태코드,상세영업상태명,폐업일자,휴업시작일자,...,사업장명,최종수정일자,데이터갱신구분,데이터갱신일자,업태구분명,좌표정보(X),좌표정보(Y),점포구분명,lon,lat
0,3000000,2004300007607500017,20040630,NaN,1,영업/정상,1,정상영업,NaN,NaN,...,동대문신발도매상가,20070721102756,I,2018-08-31 23:59:59.0,그 밖의 대규모점포,NaN,NaN,NaN,127.010641054211,37.5700690573243
1,3000000,2005300007607500001,20050127,NaN,1,영업/정상,1,정상영업,NaN,NaN,...,종로세운상가,20070721102756,I,2018-08-31 23:59:59.0,대형마트,NaN,NaN,NaN,126.995238594891,37.5692909807898
2,3010000,2014301013007500017,19960831,NaN,1,영업/정상,1,정상영업,NaN,NaN,...,현대시티타워,20170508135141,I,2018-08-31 23:59:59.0,쇼핑센터,NaN,NaN,대규모점포,127.007732349005,37.5686434582653
3,3010000,2017301013007500002,20171031,NaN,1,영업/정상,1,정상영업,NaN,NaN,...,지에스수퍼마켓 중구만리점,20180123132504,I,2018-08-31 23:59:59.0,구분없음,NaN,NaN,준대규모점포,126.965270178104,37.5541725606871
4,3070000,2000307009907500015,19951116,NaN,1,영업/정상,1,정상영업,NaN,NaN,...,동소문스카이프라자,20070707104737,I,2018-08-31 23:59:59.0,그 밖의 대규모점포,NaN,NaN,NaN,127.010321322145,37.5941783218836


In [317]:
no_mart[no_mart['lon']=='획득불가'].loc[:,['지번주소', '사업장명']]

,지번주소,사업장명


- 모두 부여 완료

#### (2) 좌표정보 있음

In [321]:
yes_mart = mart_active[mart_active['좌표정보(X)'].notnull()].reset_index(drop=True)

In [322]:
yes_mart.head()

,개방자치단체코드,관리번호,인허가일자,인허가취소일자,영업상태코드,영업상태명,상세영업상태코드,상세영업상태명,폐업일자,휴업시작일자,...,도로명주소,도로명우편번호,사업장명,최종수정일자,데이터갱신구분,데이터갱신일자,업태구분명,좌표정보(X),좌표정보(Y),점포구분명
0,3000000,2019300020207500001,20190917,NaN,1,영업/정상,1,정상영업,NaN,NaN,...,서울특별시 종로구 삼일대로 428 낙원상가 506호 (낙원동),3140.0,낙원상가,20190925150602,U,2019-09-27 02:40:00.0,그 밖의 대규모점포,198838.912966,452214.210962,대규모점포
1,3000000,2019300020207500002,20190917,NaN,1,영업/정상,1,정상영업,NaN,NaN,...,서울특별시 종로구 삼일대로 428 낙원상가 (낙원동),3140.0,낙원상가,20190925150506,I,2019-09-27 02:22:37.0,그 밖의 대규모점포,198838.912966,452214.210962,대규모점포
2,3000000,2003300012907500001,20030304,NaN,1,영업/정상,1,정상영업,NaN,NaN,...,NaN,NaN,광장주식회사,20070801173946,I,2018-08-31 23:59:59.0,시장,199837.414267,451974.128468,NaN
3,3030000,2021303010307500001,20130219,NaN,1,영업/정상,1,정상영업,NaN,NaN,...,서울특별시 성동구 왕십리로 109 성수만세주유소 (성수동1가),4768.0,홈플러스익스프레스 뚝섬2점,20210614085250,U,2020-12-05 23:06:00.0,구분없음,203841.225548,449475.540141,NaN
4,3000000,2004300007607500003,20040430,NaN,1,영업/정상,1,정상영업,NaN,NaN,...,NaN,NaN,영풍문고,20070721102756,I,2018-08-31 23:59:59.0,대형마트,198351.818737,451921.603026,NaN


In [323]:
# 좌표계를 변환할 수 있는 URL
KAKAO_URL = "https://dapi.kakao.com/v2/local/geo/transcoord.json"

# Kakao 개인 REST APP KEY
with open("./KakaoApiKey.txt", "r") as api_key:
    REST_APP_KEY = api_key.read()

In [332]:
def kakao_change(longitude=None, latitude=None, input_coord='TM', output_coord='WGS84'):
    
    url = "{}?x={}&y={}&input_coord={}&output_coord={}".format(KAKAO_URL, longitude, latitude, input_coord, output_coord)
    
    def json_request(url = url):
        
        # Headers 지정
        headers = {"Authorization": "KakaoAK {}".format(REST_APP_KEY)}
        
        # URL 정보 불러오기
        reponse = requests.get(url = url, headers = headers)
        
        return reponse.text
    
    try:
        json_response = json_request()
        json_data = json.loads(json_response)
        
        # 변환된 좌표를 얻어오는 과정에서 'x', 'y' 결과만 가져오도록 한다.
        
        json_doc = json_data.get("documents")[0]
        
        # '경도(lon)' 추출
        lon = json_doc.get("x")
        
        # '위도(lat)' 추출
        lat = json_doc.get("y")
        
    except:
        lon = "획득불가"
        lat = "획득불가"
    
    return lon, lat

In [336]:
# 'kakao_change' 함수를 사용해 '경도' 값을 추출해주는 함수 생성
def change_lon(longitude=None, latitude=None):
    
    # 'kakao_change' 함수를 사용하여 반환값 저장
    lon, lat = kakao_change(longitude, latitude)
    
    return lon

# 'kakao_change' 함수를 사용해 '위도' 값을 추출해주는 함수 생성
def change_lat(longitude=None, latitude=None):
    
    # 'kakao_change' 함수를 사용하여 반환값 저장
    lon, lat = kakao_change(longitude, latitude)
    
    return lat

In [338]:
change_lat(yes_mart.loc[0, '좌표정보(X)'], yes_mart.loc[0, '좌표정보(Y)'])

37.572217840236476

In [339]:
print("Shape of data *Before Create* :", yes_mart.shape, "\n")

# 반복문을 통해 데이터 추출
for index in tqdm(yes_mart.index, desc = "카카오 API 도로명 주소로 좌표 추출"):
    
    longitude = float(yes_mart.loc[index, '좌표정보(X)'])
    latitude = float(yes_mart.loc[index, '좌표정보(Y)'])
        
    lon = change_lon(longitude, latitude)
    lat = change_lat(longitude, latitude)
    
    yes_mart.loc[index, "lon"] = lon
    yes_mart.loc[index, "lat"] = lat

print()
print("Shape of data *After Create* :", yes_mart.shape, "\n")
display(yes_mart.head())

Shape of data *Before Create* : (657, 26) 



카카오 API 도로명 주소로 좌표 추출:   0%|          | 0/657 [00:00<?, ?it/s]


Shape of data *After Create* : (657, 28) 



,개방자치단체코드,관리번호,인허가일자,인허가취소일자,영업상태코드,영업상태명,상세영업상태코드,상세영업상태명,폐업일자,휴업시작일자,...,사업장명,최종수정일자,데이터갱신구분,데이터갱신일자,업태구분명,좌표정보(X),좌표정보(Y),점포구분명,lon,lat
0,3000000,2019300020207500001,20190917,NaN,1,영업/정상,1,정상영업,NaN,NaN,...,낙원상가,20190925150602,U,2019-09-27 02:40:00.0,그 밖의 대규모점포,198838.912966,452214.210962,대규모점포,126.987645,37.572218
1,3000000,2019300020207500002,20190917,NaN,1,영업/정상,1,정상영업,NaN,NaN,...,낙원상가,20190925150506,I,2019-09-27 02:22:37.0,그 밖의 대규모점포,198838.912966,452214.210962,대규모점포,126.987645,37.572218
2,3000000,2003300012907500001,20030304,NaN,1,영업/정상,1,정상영업,NaN,NaN,...,광장주식회사,20070801173946,I,2018-08-31 23:59:59.0,시장,199837.414267,451974.128468,NaN,126.998948,37.570056
3,3030000,2021303010307500001,20130219,NaN,1,영업/정상,1,정상영업,NaN,NaN,...,홈플러스익스프레스 뚝섬2점,20210614085250,U,2020-12-05 23:06:00.0,구분없음,203841.225548,449475.540141,NaN,127.044256,37.547536
4,3000000,2004300007607500003,20040430,NaN,1,영업/정상,1,정상영업,NaN,NaN,...,영풍문고,20070721102756,I,2018-08-31 23:59:59.0,대형마트,198351.818737,451921.603026,NaN,126.982132,37.569581


In [341]:
yes_mart[yes_mart['lon']=='획득불가']

,개방자치단체코드,관리번호,인허가일자,인허가취소일자,영업상태코드,영업상태명,상세영업상태코드,상세영업상태명,폐업일자,휴업시작일자,...,사업장명,최종수정일자,데이터갱신구분,데이터갱신일자,업태구분명,좌표정보(X),좌표정보(Y),점포구분명,lon,lat


In [353]:
yes_mart[yes_mart['lat']=='획득불가']

,개방자치단체코드,관리번호,인허가일자,인허가취소일자,영업상태코드,영업상태명,상세영업상태코드,상세영업상태명,폐업일자,휴업시작일자,...,사업장명,최종수정일자,데이터갱신구분,데이터갱신일자,업태구분명,좌표정보(X),좌표정보(Y),점포구분명,lon,lat


- 변환된 좌표를 포함한 최종 데이터 생성

In [344]:
big_mart = pd.concat([yes_mart, no_mart], axis=0).reset_index(drop=True)

In [346]:
big_mart.to_csv('./big_mart.csv', index=False, encoding='euc-kr')

### 4. 서울시 우리마을가게 상권분석서비스 상권 코드별 좌표 추출

In [356]:
commercial.head()

,기준_년월_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,엑스좌표_값,와이좌표_값,시군구_코드,행정동_코드,형태정보
0,201810,A,골목상권,1000275,보문로26길,201886,454183,11290,11290600,NaN
1,201810,A,골목상권,1000276,보문로31길,201385,454290,11290,11290555,NaN
2,201810,D,발달상권,1001045,동작구 총신대입구역_1,198304,442948,11590,11590630,NaN
3,201810,D,발달상권,1001046,서울 관악구 신림역_4,193828,443001,11620,11620695,NaN
4,201810,D,발달상권,1001047,뱅뱅사거리_1,202960,443242,11680,11680655,NaN


In [357]:
print("Shape of data *Before Create* :", commercial.shape, "\n")

# 반복문을 통해 데이터 추출
for index in tqdm(commercial.index, desc = "카카오 API 도로명 주소로 좌표 추출"):
    
    longitude = commercial.loc[index, '엑스좌표_값']
    latitude = commercial.loc[index, '와이좌표_값']
        
    lon = change_lon(longitude, latitude)
    lat = change_lat(longitude, latitude)
    
    commercial.loc[index, "lon"] = lon
    commercial.loc[index, "lat"] = lat

print()
print("Shape of data *After Create* :", commercial.shape, "\n")
display(commercial.head())

Shape of data *Before Create* : (1496, 10) 



카카오 API 도로명 주소로 좌표 추출:   0%|          | 0/1496 [00:00<?, ?it/s]


Shape of data *After Create* : (1496, 12) 



,기준_년월_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,엑스좌표_값,와이좌표_값,시군구_코드,행정동_코드,형태정보,lon,lat
0,201810,A,골목상권,1000275,보문로26길,201886,454183,11290,11290600,NaN,127.022142,37.589955
1,201810,A,골목상권,1000276,보문로31길,201385,454290,11290,11290555,NaN,127.016470,37.590920
2,201810,D,발달상권,1001045,동작구 총신대입구역_1,198304,442948,11590,11590630,NaN,126.981612,37.488729
3,201810,D,발달상권,1001046,서울 관악구 신림역_4,193828,443001,11620,11620695,NaN,126.931001,37.489187
4,201810,D,발달상권,1001047,뱅뱅사거리_1,202960,443242,11680,11680655,NaN,127.034259,37.491375


In [358]:
commercial[commercial['lon']=='획득불가']

,기준_년월_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,엑스좌표_값,와이좌표_값,시군구_코드,행정동_코드,형태정보,lon,lat


In [359]:
commercial[commercial['lat']=='획득불가']

,기준_년월_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,엑스좌표_값,와이좌표_값,시군구_코드,행정동_코드,형태정보,lon,lat


In [360]:
commercial.to_csv('./commercial_code.csv', encoding='euc-kr', index=False)